In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
cv2.bitwise_not(1)

In [ ]:
cv2.bitwise_not(2)

In [ ]:
cv2.bitwise_not(3)

In [ ]:
cv2.bitwise_not(np.array([1,2,3]))

In [ ]:
np.array([1,2,3]).dtype

In [ ]:
import sys

In [ ]:
sys.getsizeof(1)

In [ ]:
for i in range(10):
    print(f"{sys.getsizeof(i) = }")

<https://stackoverflow.com/questions/10365624/sys-getsizeofint-returns-an-unreasonably-large-value>

In [ ]:
a = cv2.bitwise_not(1)
a.shape, a.dtype

In [ ]:
a = cv2.bitwise_not(0)
a.shape, a.dtype

**(?)** So maybe `cv2.bitwise_not` only accept to deal with (arrays of) boolean values?

In [ ]:
cv2.bitwise_not(True)

In [ ]:
cv2.bitwise_not(False)

In [ ]:
True == 1

In [ ]:
False == 0

In [ ]:
np.array([0, 1], dtype=bool)

In [ ]:
import traceback

In [ ]:
try:
    cv2.bitwise_not(np.array([0, 1], dtype=bool))
except Exception:
    traceback.print_exc()

In [ ]:
np.array([False, True])

In [ ]:
try:
    cv2.bitwise_not(np.array([False, True]))
except Exception:
    traceback.print_exc()

In [ ]:
help(cv2.bitwise_not)

In [ ]:
cv2.bitwise_not(np.arange(5, dtype=np.int8))

In [ ]:
cv2.bitwise_not(np.arange(5, dtype=np.int32))

In [ ]:
cv2.bitwise_not(np.array([0,1], dtype=np.int8))

In [ ]:
cv2.bitwise_not(np.array([0,1], dtype=np.uint8))